### Import libraries

In [92]:
import os
import pandas as pd
import pyodbc, struct
from azure import identity
from faker import Faker
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel
from sqlalchemy import create_engine

from dotenv import load_dotenv
load_dotenv(override=True)
connection_string = os.environ["AZURE_SQL_CONNECTIONSTRING"]

SQL_SERVER_USERNAME = os.environ["SQL_SERVER_USERNAME"]
SQL_SERVER_ENDPOINT = os.environ["SQL_SERVER_ENDPOINT"]
SQL_SERVER_PASSWORD = os.environ["SQL_SERVER_PASSWORD"]
SQL_SERVER_DATABASE = os.environ["SQL_SERVER_DATABASE"] 
driver = "{ODBC Driver 18 for SQL Server}"
sqlalchemy_driver = "ODBC Driver 18 for SQL Server"

## Creatng a connection to the SQL Server
conn = pyodbc.connect(f'DRIVER={driver};SERVER={SQL_SERVER_ENDPOINT};PORT=1433;DATABASE={SQL_SERVER_DATABASE};UID={SQL_SERVER_USERNAME};PWD={SQL_SERVER_PASSWORD}')  
cursor = conn.cursor()

from openai import AzureOpenAI
aoai_client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-07-01-preview"
)

#https://github.com/Azure-Samples/openai/blob/main/End_to_end_Solutions/AOAISearchDemo/scripts/prepopulate/populate_sql.py

#### Reading input files

In [2]:
relative_path = "../../../../data/processed/files/"
files = os.listdir(relative_path)

df = pd.concat([pd.read_parquet(relative_path+file) for file in files]).reset_index(drop=True)
df['title_vector'] = df['title_vector'].apply(lambda x: x.tolist())
df['content_vector'] = df['content_vector'].apply(lambda x: x.tolist())
df['id'] = df['chunk_id']

CONTAINER_ID = df['preprocessing_pipeline'][0]

#### Save CSV files as tables

#### Create other SQL Tables

In [61]:
cursor.execute("""
CREATE TABLE Customers (
  cust_id INTEGER,
  cust_name VARCHAR(1000),
  cust_email VARCHAR(1000),
  cust_phone VARCHAR(1000),
  cust_address VARCHAR(1000),
  PRIMARY KEY (cust_id)
);

CREATE TABLE Products (
  prod_id INTEGER,
  prod_name VARCHAR(1000),
  price FLOAT,
  category VARCHAR(1000),
  PRIMARY KEY (prod_id)
);

CREATE TABLE Merchants (
  merchant_id INTEGER,
  merchant_name VARCHAR(1000),
  merchant_region VARCHAR(1000),
  merchant_address VARCHAR(1000),
  PRIMARY KEY (merchant_id)
);

CREATE TABLE Stock (
  prod_id INTEGER,
  merchant_id INTEGER,
  stock INTEGER,
  PRIMARY KEY (prod_id, merchant_id),
  FOREIGN KEY (merchant_id) REFERENCES Merchants(merchant_id),
  FOREIGN KEY (prod_id) REFERENCES Products(prod_id)
);

CREATE TABLE Sales (
    sale_id INTEGER,
    cust_id INTEGER,
    merchant_id INTEGER,
    date DATETIME,
    total_price FLOAT,
    PRIMARY KEY (sale_id),
    FOREIGN KEY (cust_id) REFERENCES Customers(cust_id),
    FOREIGN KEY (merchant_id) REFERENCES Merchants(merchant_id)
);

CREATE TABLE Sales_Detail (
  sales_id INTEGER,
  prod_id INTEGER,
  quantity INTEGER,
  PRIMARY KEY (sales_id, prod_id),
  FOREIGN KEY (sales_id) REFERENCES Sales(sale_id),
  FOREIGN KEY (prod_id) REFERENCES Products(prod_id)
);
""")

cursor.commit()

#### Inserting Tables

In [ ]:
def CSVToSQL(table_name,file_location):
    # Read the CSV file
    df = pd.read_csv(file_location)
    for index, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(df.columns)})
        VALUES ({', '.join(['?' for _ in df.columns])})
        """
        cursor.execute(insert_query, tuple(row))
    conn.commit()

In [62]:
relative_path = "../../../../data/processed/structured/"
files = os.listdir(relative_path)

CSVToSQL("Customers",relative_path+'/customers.csv')
CSVToSQL("Products",relative_path+'/products.csv')
CSVToSQL("Merchants",relative_path+'/merchants.csv')
CSVToSQL("Stock",relative_path+'/stock.csv')
CSVToSQL("Sales",relative_path+'/sales.csv')
CSVToSQL("Sales_Detail",relative_path+'/sales_detail.csv')

cursor.commit()

### Generating description this data

In [90]:
string_to_description = ""
for file in files:
    table_name = file.replace(".csv","")
    values = str(pd.read_csv(relative_path+file).head().to_dict(orient='records'))
    sample_string = "Table name: " + table_name + " \n" + "Values: " + values+ " \n"
    string_to_description += sample_string

In [91]:
print(string_to_description)

Table name: customers 
Values: [{'cust_id': 1, 'cust_name': 'John Doe', 'cust_email': 'john.doe@example.com', 'cust_phone': '555-123-4567', 'cust_address': '123 Main St, City A'}, {'cust_id': 2, 'cust_name': 'Jane Smith', 'cust_email': 'jane.smith@example.com', 'cust_phone': '555-987-6543', 'cust_address': '456 High St, City B'}, {'cust_id': 3, 'cust_name': 'Michael Johnson', 'cust_email': 'michael.johnson@example.com', 'cust_phone': '555-456-7890', 'cust_address': '789 First St, City C'}, {'cust_id': 4, 'cust_name': 'Sara Williams', 'cust_email': 'sara.williams@example.com', 'cust_phone': '555-321-6549', 'cust_address': '147 Second St, City D'}, {'cust_id': 5, 'cust_name': 'Kevin Brown', 'cust_email': 'kevin.brown@example.com', 'cust_phone': '555-987-3214', 'cust_address': '258 Third St, City E'}] 
Table name: merchants 
Values: [{'merchant_id': 1, 'merchant_name': 'ElectroMegaMart', 'merchant_region': 'North America', 'merchant_address': '123 Tech Plaza, New York, NY 10001'}, {'merch

In [156]:
def GetToolDescription(string_to_description):
    messages = [{"role":"system","content":"You are an agent that takes information about tables, including table names and sample values, with the objective of generating a description that will be used to select an OpenAI tool that will help users to interact with the mentioned tables. Please provide examples of each field"},
            {"role":"user", "content":"Information about tables "+ string_to_description}]

    response = aoai_client.chat.completions.create(model="gpt4o",  
                                    messages = messages, 
                                    temperature=0.2,  
                                    max_tokens=2000,
                                    seed = 42)
    DBDescription = response.choices[0].message.content
    print(DBDescription)

    return DBDescription

In [157]:
tool_description = GetToolDescription(string_to_description)

The database consists of six tables: **customers**, **merchants**, **products**, **sales**, **sales_detail**, and **stock**. 

1. **Customers Table**: This table contains customer information such as ID, name, email, phone number, and address. For example, customer ID 1 is John Doe with the email john.doe@example.com and resides at 123 Main St, City A.

2. **Merchants Table**: This table holds merchant details including ID, name, region, and address. For instance, merchant ID 1 is ElectroMegaMart located at 123 Tech Plaza, New York, NY 10001.

3. **Products Table**: This table lists products with details like product ID, name, price, and category. An example is product ID 1, Surface Pro 9, priced at $999.99 under the Tablet category.

4. **Sales Table**: This table records sales transactions with fields for sale ID, customer ID, merchant ID, date, and total price. For example, sale ID 1 involves customer ID 8 and merchant ID 9 on 2023-01-01 with a total price of $75.

5. **Sales Detail

#### Saving description to file

In [147]:
with open("../../../../data/processed/tool_description/GetSQLDBAnswerDescription.txt", "w") as file:
    # Write the string to the file
    file.write(tool_description)

In [148]:
result = cursor.execute("SELECT prod_name, category, price FROM Products WHERE prod_name like '%Surface%' and category like '%accessory%';")
[row for row in result.fetchall()]

[('Surface Slim Pen 2', 'Accessory', 94.99),
 ('Surface Pen', 'Accessory', 99.99),
 ('Surface Dock', 'Accessory', 199.99)]

In [149]:
result = cursor.execute("SELECT SUM(stock) AS total_stock FROM Stock WHERE prod_id IN (SELECT prod_id FROM Products WHERE prod_name LIKE '%Surface Laptop 5%');")
[row for row in result.fetchall()]

[(2527,)]